In [24]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os
from os.path import join
import sys
import json
import geopandas as gpd

In [3]:
# Load the "autoreload" extension
%load_ext autoreload

# always reload modules marked with "%aimport"
%autoreload 1

In [25]:
# add the 'src' directory as one where we can import modules
src_dir = join(os.getcwd(), os.pardir, 'src')
sys.path.append(src_dir)

In [90]:
%aimport Data.make_data
from Data.make_data import states_in_nerc
%aimport Analysis.index
from Analysis.index import facility_emission_gen, group_facility_data
%aimport Analysis.index
from Analysis.index import facility_co2, adjust_epa_emissions, group_fuel_cats
%aimport util.utils
from util.utils import rename_cols, add_facility_location

## Load data

In [78]:
cwd = os.getcwd()
path = join(cwd, '..', 'Data storage',
            'Facility gen fuels and CO2 2017-08-31.zip')
eia_fac = pd.read_csv(path)

In [79]:
rename_cols(eia_fac)

In [80]:
eia_fac.head()

,f,fuel,month,plant id,total fuel (mmbtu),year,generation (mwh),elec fuel (mmbtu),geography,last_updated,lat,lon,prime mover,datetime,quarter,all fuel fossil co2 (kg),elec fuel fossil co2 (kg),all fuel total co2 (kg),elec fuel total co2 (kg)
0,M,SUB,6,10360,0.0,2017,0.0,0.0,USA-WI,2017-08-24T11:46:12-04:00,44.4936,-88.0303,ALL,2017-06-01,2,0.0,0.0,0.0,0.0
1,M,SUB,5,10360,0.0,2017,0.0,0.0,USA-WI,2017-08-24T11:46:12-04:00,44.4936,-88.0303,ALL,2017-05-01,2,0.0,0.0,0.0,0.0
2,M,SUB,4,10360,0.0,2017,0.0,0.0,USA-WI,2017-08-24T11:46:12-04:00,44.4936,-88.0303,ALL,2017-04-01,2,0.0,0.0,0.0,0.0
3,M,SUB,3,10360,0.0,2017,0.0,0.0,USA-WI,2017-08-24T11:46:12-04:00,44.4936,-88.0303,ALL,2017-03-01,1,0.0,0.0,0.0,0.0
4,M,SUB,2,10360,0.0,2017,0.0,0.0,USA-WI,2017-08-24T11:46:12-04:00,44.4936,-88.0303,ALL,2017-02-01,1,0.0,0.0,0.0,0.0


In [48]:
path = join(cwd, '..', 'Data storage', 'Derived data',
            'Monthly EPA emissions 2017-08-31.csv')
epa = pd.read_csv(path)

## Test facility_emissions_gen

In [49]:
fuel_cat_folder = join(cwd, '..', 'Data storage', 'Fuel categories')
state_cats_path = join(fuel_cat_folder, 'State_facility.json')

with open(state_cats_path, 'r') as f:
    state_fuel_cat = json.load(f)
    
custom_cats_path = join(fuel_cat_folder, 'Custom_results.json')
with open(custom_cats_path, 'r') as f:
    custom_fuel_cat = json.load(f)

In [32]:
co2, gen_fuels_custom = facility_emission_gen(eia_facility=eia_fac, epa=epa,
                                              state_fuel_cat=state_fuel_cat,
                                              custom_fuel_cat=custom_fuel_cat,
                                              export_state_cats=False)

Renaming columns
Grouping facilities
Adjusting EPA emissions
Caculating CO2
Gen/fuels to state categories


In [50]:
co2, gen_fuels_state = facility_emission_gen(eia_facility=eia_fac, epa=epa,
                                              state_fuel_cat=state_fuel_cat,
                                              custom_fuel_cat=custom_fuel_cat,
                                              export_state_cats=True)

Renaming columns
Grouping facilities
Adjusting EPA emissions
Caculating CO2
Gen/fuels to state categories


In [51]:
co2.head()

,year,month,plant id,final co2 (kg)
0,2001,1,2,0.000000e+00
1,2001,1,3,9.625411e+08
2,2001,1,4,0.000000e+00
3,2001,1,7,6.700092e+07
4,2001,1,8,7.530204e+08


In [199]:
gen_fuels_state.head()

,type,year,month,plant id,total fuel (mmbtu),generation (mwh),elec fuel (mmbtu),lat,lon,quarter,all fuel fossil co2 (kg),elec fuel fossil co2 (kg),all fuel total co2 (kg),elec fuel total co2 (kg)
0,COW,2001,1,3,8275496.0,852306.0,8275496.0,31.006900,-88.010300,1,772103776.8,772103776.8,772103776.8,772103776.8
1,COW,2001,1,7,587162.0,45991.0,587162.0,34.012800,-85.970800,1,54782214.6,54782214.6,54782214.6,54782214.6
2,COW,2001,1,8,6910786.0,706857.0,6910786.0,33.644344,-87.196486,1,644776333.8,644776333.8,644776333.8,644776333.8
3,COW,2001,1,10,3108892.0,320454.0,3108892.0,32.601700,-87.781100,1,290059623.6,290059623.6,290059623.6,290059623.6
4,COW,2001,1,26,9745127.0,1005460.0,9745127.0,33.244211,-86.458056,1,909220349.1,909220349.1,909220349.1,909220349.1


### Check generation and fuel consumption totals

Interesting - there is some small part of generation that I'm losing along the way. It's 5 orders of magnitude smaller than the total though.

In [121]:
eia_fac['generation (mwh)'].sum()

65594379743.76258

In [123]:
gen_fuels_state['generation (mwh)'].sum()

65594327524.76352

Fuel consumption is pretty identical though

In [124]:
eia_fac['total fuel (mmbtu)'].sum(), eia_fac['elec fuel (mmbtu)'].sum()

(686393145435.8103, 645653921528.091)

In [125]:
gen_fuels_state['total fuel (mmbtu)'].sum(), gen_fuels_state['elec fuel (mmbtu)'].sum()

(686393145435.7861, 645653921528.0757)

## Extra gen/fuels from non-reporting

In [200]:
%aimport Analysis.index
from Analysis.index import extra_emissions_gen

Total EIA generation/fuel consumption and emission factors

In [201]:
cwd = os.getcwd()
path = join(cwd, '..', 'Data storage',
            'EIA country-wide gen fuel CO2 2017-08-31.csv')
eia_total = pd.read_csv(path)

path = join(cwd, '..', 'Data storage',
            'Final emission factors.csv')
ef = pd.read_csv(path, index_col=0)

In [202]:
eia_total.head()

,end,f,geography,last_updated,sector,series_id,start,type,units,year,month,generation (MWh),datetime,quarter,total fuel (mmbtu),elec fuel (mmbtu),all fuel CO2 (kg),elec fuel CO2 (kg)
0,201706,M,USA,2017-08-24T11:46:12-04:00,99,ELEC.GEN.DPV-US-99.M,201401,DPV,thousand megawatthours,2017,6,2524250.11,2017-06-01,2,NaN,NaN,0.0,0.0
1,201706,M,USA,2017-08-24T11:46:12-04:00,99,ELEC.GEN.DPV-US-99.M,201401,DPV,thousand megawatthours,2017,5,2479480.14,2017-05-01,2,NaN,NaN,0.0,0.0
2,201706,M,USA,2017-08-24T11:46:12-04:00,99,ELEC.GEN.DPV-US-99.M,201401,DPV,thousand megawatthours,2017,4,2267811.28,2017-04-01,2,NaN,NaN,0.0,0.0
3,201706,M,USA,2017-08-24T11:46:12-04:00,99,ELEC.GEN.DPV-US-99.M,201401,DPV,thousand megawatthours,2017,3,2054274.30,2017-03-01,1,NaN,NaN,0.0,0.0
4,201706,M,USA,2017-08-24T11:46:12-04:00,99,ELEC.GEN.DPV-US-99.M,201401,DPV,thousand megawatthours,2017,2,1449156.85,2017-02-01,1,NaN,NaN,0.0,0.0


In [136]:
extra_co2, extra_gen_fuel = extra_emissions_gen(gen_fuels_state, eia_total, ef)

Results match what I have previously found in the notebooks up on GitHub (Emissions Index repo)

In [94]:
extra_gen_fuel.head()

total fuel (mmbtu)  generation (mwh)  elec fuel (mmbtu)
type year month                                                         
COW  2001 1              16918390.0       1011665.061         13157576.0
          2              14968346.0        911346.865         11574491.0
          3              18321543.0       1153328.913         14765162.0
          4              13530117.0        776341.623         10374008.0
          5              14014024.0        801266.923         10920463.0

In [95]:
eia_extra.loc[idx['DPV',:,:]]

total fuel (mmbtu)  generation (mwh)  elec fuel (mmbtu)
type year month                                                         
DPV  2014 1                     NaN         624032.82                NaN
          2                     NaN         663828.73                NaN
          3                     NaN         907029.95                NaN
          4                     NaN         988423.77                NaN
          5                     NaN        1092011.80                NaN
          6                     NaN        1100560.82                NaN
          7                     NaN        1148527.79                NaN
          8                     NaN        1139134.66                NaN
          9                     NaN        1046360.70                NaN
          10                    NaN         964877.96                NaN
          11                    NaN         791801.55                NaN
          12                    NaN         766068.15                NaN
     2015 1                     NaN         746445.84                NaN
          2                     NaN         815764.50                NaN
          3                     NaN        1133577.23                NaN
          4                     NaN        1263528.30                NaN
          5                     NaN        1393836.75                NaN
          6                     NaN        1408013.52                NaN
          7                     NaN        1486681.59                NaN
          8                     NaN        1468048.33                NaN
          9                     NaN        1329651.46                NaN
          10                    NaN        1197707.54                NaN
          11                    NaN         982048.27                NaN
          12                    NaN         913963.95                NaN
     2016 1                     NaN        1020588.78                NaN
          2                     NaN        1189469.53                NaN
          3                     NaN        1582361.09                NaN
          4                     NaN        1763404.74                NaN
          5                     NaN        1944571.47                NaN
          6                     NaN        1991456.73                NaN
          7                     NaN        2065890.59                NaN
          8                     NaN        2003636.20                NaN
          9                     NaN        1789836.27                NaN
          10                    NaN        1607185.85                NaN
          11                    NaN        1306748.99                NaN
          12                    NaN        1201677.95                NaN
     2017 1                     NaN        1282339.18                NaN
          2                     NaN        1449156.85                NaN
          3                     NaN        2054274.30                NaN
          4                     NaN        2267811.28                NaN
          5                     NaN        2479480.14                NaN
          6                     NaN        2524250.11                NaN

## Total CO₂ (national)

In [130]:
facility_co2 = co2.groupby(['year', 'month']).sum()

In [131]:
facility_co2.tail()

plant id  final co2 (kg)
year month                          
2017 2      57482310    1.165442e+11
     3      58485598    1.237187e+11
     4      58955690    1.133454e+11
     5      59223601    1.292581e+11
     6      59451819    1.512113e+11

In [141]:
extra_co2.loc[idx['NG', :, :],:].tail(n=11)

all fuel co2 (kg)  elec fuel co2 (kg)
type year month                                       
NG   2016 8           6.022574e+09        4.039321e+09
          9           4.917984e+09        3.118869e+09
          10          4.312456e+09        2.624005e+09
          11          4.231444e+09        2.458983e+09
          12          4.418713e+09        2.530048e+09
     2017 1           7.823823e+09        6.101011e+09
          2           6.601814e+09        5.030973e+09
          3           8.630216e+09        6.986014e+09
          4           7.823192e+09        6.280070e+09
          5           8.749634e+09        7.244919e+09
          6           1.000377e+10        8.465898e+09

In [133]:
national_co2 = (facility_co2.loc[:, 'final co2 (kg)']
                + extra_co2.loc[:, 'elec fuel co2 (kg)'])

These values are very close, but not quite exactly the same as what I've put up on the website.

Why? Could it be because the extra 

In [134]:
national_co2

year  month
2001  1        2.162121e+11
      2        1.814475e+11
      3        1.900841e+11
      4        1.761317e+11
      5        1.905779e+11
      6        2.062662e+11
      7        2.311815e+11
      8        2.404311e+11
      9        1.950060e+11
      10       1.845210e+11
      11       1.731834e+11
      12       1.882633e+11
2002  1        1.951862e+11
      2        1.709727e+11
      3        1.855136e+11
      4        1.759043e+11
      5        1.863580e+11
      6        2.081023e+11
      7        2.385440e+11
      8        2.355222e+11
      9        2.092338e+11
      10       1.948277e+11
      11       1.844629e+11
      12       2.023788e+11
2003  1        2.167834e+11
      2        1.880347e+11
      3        1.876779e+11
      4        1.727763e+11
      5        1.840170e+11
      6        2.007602e+11
                   ...     
2015  1        1.779027e+11
      2        1.696220e+11
      3        1.522496e+11
      4        1.310557e+11
      5 

## Facility state and lat/lon file generation

In [148]:
%aimport Data.make_data
from Data.make_data import facility_location_data

In [150]:
facility_location_data(eia_fac)

## Fraction of estimated gen/fuels in each NERC region

**NOTE** A few of the 2015 annual facilities get assigned multiple NERC regions. I'm not sure why this is. A test of generation from these facilities showed that it was several orders of magnitude lower than total generation from all 2015 annual reporting facilities. 

In [9]:
%aimport Data.make_data
from Data.make_data import get_annual_plants

In [10]:
annual_ids = get_annual_plants(2015)

In [14]:
annual_2015 = eia_fac.loc[(eia_fac['year'] == 2015) &
                          (eia_fac['plant id'].isin(annual_ids)), :].copy()

In [15]:
annual_2015['generation (mwh)'].sum()

210166168.213

### Add NERC region labels

In [33]:
%aimport Analysis.state2nerc
from Analysis.state2nerc import fraction_state2nerc, add_region

In [22]:
cwd = os.getcwd()
path = join(cwd, '..', 'Data storage', 'NERC_Regions_EIA',
            'NercRegions_201610.shp')
regions = gpd.read_file(path)

In [107]:
regions.head()

,NERC,NERC_Label,geometry
0,-,Indeterminate - various NERC membership,(POLYGON ((-91.71222639199999 32.9756623780000...
1,FRCC,Florida Reliability Coordinating Council (FRCC),(POLYGON ((-81.95506602699999 24.5196900770000...
2,MRO,Midwest Reliability Organization (MRO),POLYGON ((-95.07148604699995 49.36451082200006...
3,NPCC,Northeast Power Coordinating Council (NPCC),(POLYGON ((-73.82134384999995 40.6045004290000...
4,RFC,ReliabilityFirst Corporation (RFC),(POLYGON ((-90.89359863199996 29.0467829150000...


In [91]:
cwd = os.getcwd()
path = join(cwd, '..', 'Data storage', 'Facility labels',
            'Facility locations.csv')
location_labels = pd.read_csv(path)

In [36]:
#Drop facilities without lat/lon data
eia_fac_nerc = eia_fac.dropna(subset=['lat', 'lon'])

In [38]:
eia_fac_nerc = add_region(eia_fac_nerc, regions, region_col='nerc')
eia_fac_nerc['state'] = eia_fac_nerc['geography'].str[-2:]

In [55]:
eia_fac_nerc_type = group_fuel_cats(eia_fac_nerc, state_fuel_cat)

In [58]:
eia_fac_nerc_type.columns

Index(['type', 'year', 'month', 'plant id', 'total fuel (mmbtu)',
       'generation (MWh)', 'elec fuel (mmbtu)', 'lat', 'lon', 'quarter',
       'all fuel fossil CO2 (kg)', 'elec fuel fossil CO2 (kg)',
       'all fuel total CO2 (kg)', 'elec fuel total CO2 (kg)', 'nerc'],
      dtype='object')

In [18]:
nerc_state_path = join(cwd, '..', 'Data storage', 'Derived data',
                  'NERC_states.json')

with open(nerc_state_path, 'r') as f:
    nerc_states = json.load(f)

In [160]:
eia_2015_annual = eia_fac.loc[eia_fac['plant id'].isin(annual_ids)].copy()
eia_2015_annual.head()

,f,fuel,month,plant id,total fuel (mmbtu),year,generation (mwh),elec fuel (mmbtu),geography,last_updated,lat,lon,prime mover,datetime,quarter,all fuel fossil co2 (kg),elec fuel fossil co2 (kg),all fuel total co2 (kg),elec fuel total co2 (kg)
120,M,NG,12,10052,25674.0,2015,1290.744,25674.0,USA-CA,2017-08-24T11:46:12-04:00,40.7995,-124.2028,ALL,2015-12-01,4,1362519.18,1362519.18,1362519.18,1362519.18
121,M,NG,11,10052,6341.0,2015,378.814,6341.0,USA-CA,2017-08-24T11:46:12-04:00,40.7995,-124.2028,ALL,2015-11-01,4,336516.87,336516.87,336516.87,336516.87
122,M,NG,10,10052,2898.0,2015,183.916,2898.0,USA-CA,2017-08-24T11:46:12-04:00,40.7995,-124.2028,ALL,2015-10-01,4,153796.86,153796.86,153796.86,153796.86
123,M,NG,9,10052,2947.0,2015,191.471,2947.0,USA-CA,2017-08-24T11:46:12-04:00,40.7995,-124.2028,ALL,2015-09-01,3,156397.29,156397.29,156397.29,156397.29
124,M,NG,8,10052,1761.0,2015,117.711,1761.0,USA-CA,2017-08-24T11:46:12-04:00,40.7995,-124.2028,ALL,2015-08-01,3,93456.27,93456.27,93456.27,93456.27


In [161]:
eia_2015_annual = group_fuel_cats(eia_2015_annual, state_fuel_cat)
eia_2015_annual.head()

,type,year,month,plant id,total fuel (mmbtu),generation (mwh),elec fuel (mmbtu),lat,lon,quarter,all fuel fossil co2 (kg),elec fuel fossil co2 (kg),all fuel total co2 (kg),elec fuel total co2 (kg)
0,COW,2001,1,7,587162.0,45991.000,587162.0,34.012800,-85.970800,1,54782214.6,54782214.6,54782214.6,54782214.6
1,COW,2001,1,59,674160.0,62539.000,674160.0,40.854765,-98.348222,1,65528352.0,65528352.0,65528352.0,65528352.0
2,COW,2001,1,642,337326.0,27208.000,337326.0,30.669200,-84.886900,1,31472515.8,31472515.8,31472515.8,31472515.8
3,COW,2001,1,753,3794.0,118.512,3794.0,31.844708,-83.940734,1,353980.2,353980.2,353980.2,353980.2
4,COW,2001,1,964,309027.0,23207.000,309027.0,39.757327,-89.600512,1,28832219.1,28832219.1,28832219.1,28832219.1


In [162]:
eia_2015_annual = add_facility_location(eia_2015_annual, location_labels, labels=['state'])
eia_2015_annual.head()

,type,year,month,plant id,total fuel (mmbtu),generation (mwh),elec fuel (mmbtu),lat,lon,quarter,all fuel fossil co2 (kg),elec fuel fossil co2 (kg),all fuel total co2 (kg),elec fuel total co2 (kg),state
0,COW,2001,1,7,587162.0,45991.0,587162.0,34.0128,-85.9708,1,54782214.6,54782214.6,54782214.6,54782214.6,AL
1,COW,2001,2,7,463753.0,33025.0,463753.0,34.0128,-85.9708,1,43268154.9,43268154.9,43268154.9,43268154.9,AL
2,COW,2001,3,7,562534.0,37340.0,562534.0,34.0128,-85.9708,1,52484422.2,52484422.2,52484422.2,52484422.2,AL
3,COW,2001,4,7,600426.0,43914.0,600426.0,34.0128,-85.9708,2,56019745.8,56019745.8,56019745.8,56019745.8,AL
4,COW,2001,5,7,489574.0,35148.0,489574.0,34.0128,-85.9708,2,45677254.2,45677254.2,45677254.2,45677254.2,AL


In [163]:
eia_2015_annual_nerc = add_region(eia_2015_annual,
                             regions, region_col='nerc')
eia_2015_annual_nerc.head()

,type,year,month,plant id,total fuel (mmbtu),generation (mwh),elec fuel (mmbtu),lat,lon,quarter,all fuel fossil co2 (kg),elec fuel fossil co2 (kg),all fuel total co2 (kg),elec fuel total co2 (kg),state,nerc
0,COW,2001,1,7,587162.0,45991.0,587162.0,34.0128,-85.9708,1,54782214.6,54782214.6,54782214.6,54782214.6,AL,SERC
1,COW,2001,2,7,463753.0,33025.0,463753.0,34.0128,-85.9708,1,43268154.9,43268154.9,43268154.9,43268154.9,AL,SERC
2,COW,2001,3,7,562534.0,37340.0,562534.0,34.0128,-85.9708,1,52484422.2,52484422.2,52484422.2,52484422.2,AL,SERC
3,COW,2001,4,7,600426.0,43914.0,600426.0,34.0128,-85.9708,2,56019745.8,56019745.8,56019745.8,56019745.8,AL,SERC
4,COW,2001,5,7,489574.0,35148.0,489574.0,34.0128,-85.9708,2,45677254.2,45677254.2,45677254.2,45677254.2,AL,SERC


In [188]:
all_states = []
for value in nerc_states.values():
    all_states.extend(value)
all_states = set(all_states)

In [189]:
all_states

{'AL',
 'AR',
 'AZ',
 'CA',
 'CO',
 'CT',
 'DC',
 'DE',
 'FL',
 'GA',
 'IA',
 'ID',
 'IL',
 'IN',
 'KS',
 'KY',
 'LA',
 'MA',
 'MD',
 'ME',
 'MI',
 'MN',
 'MO',
 'MS',
 'MT',
 'NC',
 'ND',
 'NE',
 'NH',
 'NJ',
 'NM',
 'NV',
 'NY',
 'OH',
 'OK',
 'OR',
 'PA',
 'RI',
 'SC',
 'SD',
 'TN',
 'TX',
 'UT',
 'VA',
 'VT',
 'WA',
 'WI',
 'WV',
 'WY'}

In [398]:
df_list = []
for state in all_states:
    try:
        df_list.append(fraction_state2nerc(eia_2015_annual_nerc,
                       state, region_col='nerc', fuel_col='type'))
    except:
        print(state)
        pass

DC


In [402]:
nerc_fractions = pd.concat(df_list)
nerc_fractions.set_index(['state', 'nerc', 'type'], inplace=True)
nerc_fractions.sort_index(inplace=True)
nerc_fractions.head()

% generation  % total fuel  % elec fuel
state nerc type                                         
AL    SERC COW            1.0           1.0          1.0
           HYC            1.0           1.0          1.0
           NG             1.0           1.0          1.0
           OOG            1.0           1.0          1.0
           OTH            1.0           1.0          1.0

With the values below I can allocate extra state-level generation and fuel use to each of the NERC regions!

In [403]:
nerc_fractions.tail()

% generation  % total fuel  % elec fuel
state nerc type                                         
WY    WECC NG             1.0           1.0          1.0
           OOG            1.0           1.0          1.0
           OTH            1.0           1.0          1.0
           PEL            1.0           1.0          1.0
           WND            1.0           1.0          1.0

In [198]:
nerc_fractions.groupby(['state', 'type']).sum()

% generation  % total fuel  % elec fuel
state type                                         
AL    COW            1.0           1.0          1.0
      HYC            1.0           1.0          1.0
      NG             1.0           1.0          1.0
      OOG            1.0           1.0          1.0
      OTH            1.0           1.0          1.0
      PEL            1.0           1.0          1.0
      WAS            1.0           1.0          1.0
      WWW            1.0           1.0          1.0
AR    HYC            1.0           1.0          1.0
      NG             1.0           1.0          1.0
      OOG            1.0           1.0          1.0
      PC             1.0           1.0          1.0
      PEL            1.0           1.0          1.0
      SUN            1.0           1.0          1.0
      WAS            1.0           1.0          1.0
      WWW            1.0           1.0          1.0
AZ    HYC            1.0           1.0          1.0
      NG             1.0           1.0          1.0
      OOG            1.0           1.0          1.0
      PEL            1.0           1.0          1.0
      SUN            1.0           1.0          1.0
      WND            1.0           1.0          1.0
CA    COW            1.0           1.0          1.0
      GEO            1.0           1.0          1.0
      HYC            1.0           1.0          1.0
      NG             1.0           1.0          1.0
      OOG            1.0           1.0          1.0
      OTH            1.0           1.0          1.0
      PC             1.0           1.0          1.0
      PEL            1.0           1.0          1.0
...                  ...           ...          ...
VT    WWW            1.0           1.0          1.0
WA    HYC            1.0           1.0          1.0
      NG             1.0           1.0          1.0
      OOG            1.0           1.0          1.0
      OTH            1.0           1.0          1.0
      PEL            1.0           1.0          1.0
      WAS            1.0           1.0          1.0
      WND            1.0           1.0          1.0
      WWW            1.0           1.0          1.0
WI    COW            1.0           1.0          1.0
      HYC            1.0           1.0          1.0
      NG             1.0           1.0          1.0
      OOG            1.0           1.0          1.0
      OTH            1.0           1.0          1.0
      PEL            1.0           1.0          1.0
      SUN            1.0           1.0          1.0
      WAS            1.0           1.0          1.0
      WND            1.0           1.0          1.0
      WWW            1.0           1.0          1.0
WV    COW            1.0           1.0          1.0
      HYC            1.0           1.0          1.0
      NG             1.0           1.0          1.0
      OOG            1.0           1.0          1.0
WY    COW            1.0           1.0          1.0
      HYC            1.0           1.0          1.0
      NG             1.0           1.0          1.0
      OOG            1.0           1.0          1.0
      OTH            1.0           1.0          1.0
      PEL            1.0           1.0          1.0
      WND            1.0           1.0          1.0

[405 rows x 3 columns]

## Allocate extra gen from the state-level to regions

I still need to generate state-level total generation and fuel use!

In [239]:
# a dictionary to match column names
nerc_frac_match = {'% generation': 'generation (mwh)',
                   '% total fuel': 'total fuel (mmbtu)',
                   '% elec fuel': 'elec fuel (mmbtu)'}

In [357]:
path = join(cwd, '..', 'Data storage', 'EIA state-level gen fuel CO2.csv')

state_total = pd.read_csv(path)

In [358]:
rename_cols(state_total)

In [359]:
state_total['state'] = state_total['geography'].str[-2:]

In [360]:
state_total.head()

,end,f,geography,last_updated,sector,series_id,start,type,units,year,month,generation (mwh),datetime,quarter,total fuel (mmbtu),elec fuel (mmbtu),all fuel co2 (kg),elec fuel co2 (kg),state
0,201706,M,USA-MN,2017-08-24T11:46:12-04:00,99,ELEC.GEN.AOR-MN-99.M,200101,AOR,megawatthours,2017,6,1066984.87,2017-06-01,2.0,NaN,NaN,NaN,NaN,MN
1,201706,M,USA-MN,2017-08-24T11:46:12-04:00,99,ELEC.GEN.AOR-MN-99.M,200101,AOR,megawatthours,2017,5,1173596.09,2017-05-01,2.0,NaN,NaN,NaN,NaN,MN
2,201706,M,USA-MN,2017-08-24T11:46:12-04:00,99,ELEC.GEN.AOR-MN-99.M,200101,AOR,megawatthours,2017,4,1250625.52,2017-04-01,2.0,NaN,NaN,NaN,NaN,MN
3,201706,M,USA-MN,2017-08-24T11:46:12-04:00,99,ELEC.GEN.AOR-MN-99.M,200101,AOR,megawatthours,2017,3,1345113.65,2017-03-01,1.0,NaN,NaN,NaN,NaN,MN
4,201706,M,USA-MN,2017-08-24T11:46:12-04:00,99,ELEC.GEN.AOR-MN-99.M,200101,AOR,megawatthours,2017,2,1222310.74,2017-02-01,1.0,NaN,NaN,NaN,NaN,MN


In [361]:
cols = list(nerc_frac_match.values())
state_total = state_total.groupby(['state', 'year', 'month', 'type'])[cols].sum()
state_total.head()

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
state year month type                                                         
AK    2001 1     AOR               87.0                 NaN                NaN
                 COW            46903.0           1120000.0           872000.0
                 HYC           104549.0                 NaN                NaN
                 NG            367521.0           4091000.0          3989000.0
                 PEL            71085.0            767000.0           763000.0

In [208]:
nercs = nerc_fractions['nerc'].unique()

In [362]:
cols = list(nerc_frac_match.values())
eia_fac_type = group_fuel_cats(eia_fac, state_fuel_cat)
eia_fac_type = add_facility_location(eia_fac_type, location_labels, ['state'])
eia_fac_type = eia_fac_type.groupby(['state', 'year', 'month', 'type'])[cols].sum()

In [363]:
eia_fac_type.head()

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
state year month type                                                         
AK    2001 1     COW          26493.302           576574.00          520312.00
                 HYC         103940.302          1074015.14         1074015.14
                 NG          314206.726          3372210.00         3270620.00
                 PEL          66093.098           715111.00          710785.00
                 WND             86.518              893.99             893.99

In [438]:
state_extra = (state_total.loc[idx[:, 2016:, :, :], :]
               - eia_fac_type.loc[idx[:, 2016:, :, :], :])
state_extra.dropna(how='all', inplace=True)
state_extra = state_extra.reorder_levels(['year', 'state', 'month', 'type'])
state_extra.head()

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
year state month type                                                         
2016 AK    1     COW          20785.451            367385.0           306487.0
                 HYC          62929.960                 NaN                NaN
                 NG           68565.140            767269.0           767269.0
                 PEL          38760.835            400331.0           379272.0
           2     COW          19546.818            352264.0           294007.0

In [439]:
state_extra.loc[2016].head()

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
state month type                                                         
AK    1     COW          20785.451            367385.0           306487.0
            HYC          62929.960                 NaN                NaN
            NG           68565.140            767269.0           767269.0
            PEL          38760.835            400331.0           379272.0
      2     COW          19546.818            352264.0           294007.0

Why is generation from facilities greater than state-level estimates in some cases?

In [406]:
nerc_fractions.sort_index(inplace=True)
nerc_fractions.loc[idx['TX', 'WECC', :], :].tail()

% generation  % total fuel  % elec fuel
state nerc type                                         
TX    WECC NG        0.007022      0.007095     0.008178
           PEL       0.002196      0.000838     0.002380
           SUN       0.062262      0.061306     0.061306
           WND       0.003737      0.003891     0.003891

In [437]:
state_extra.loc[idx[:, 2016, :, :], :].head()

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
state year month type                                                         
AK    2016 1     COW          20785.451            367385.0           306487.0
                 HYC          62929.960                 NaN                NaN
                 NG           68565.140            767269.0           767269.0
                 PEL          38760.835            400331.0           379272.0
           2     COW          19546.818            352264.0           294007.0

In [436]:
state_extra_2016 = state_extra.loc[idx[:, 2016, :, :], :]
state_extra_2016.index = state_extra_2016.index.droplevel('year')

state_extra_2017 = state_extra.loc[idx[:, 2017, :, :], :]
state_extra_2017.index = state_extra_2017.index.droplevel('year')

In [388]:
state_extra_2016.head()

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
state month type                                                         
AK    1     COW          20785.451            367385.0           306487.0
            HYC          62929.960                 NaN                NaN
            NG           68565.140            767269.0           767269.0
            PEL          38760.835            400331.0           379272.0
      2     COW          19546.818            352264.0           294007.0

In [414]:
df_list = []
for month in range(1, 13):
    df = nerc_fractions.copy()
    df['month'] = month
    df.set_index('month', append=True, inplace=True)
    df_list.append(df)

nerc_frac_monthly = pd.concat(df_list, axis=0)
nerc_frac_monthly.sort_index(inplace=True)
nerc_frac_monthly = (nerc_frac_monthly
                     .reorder_levels(['nerc', 'state', 'month', 'type']))

In [417]:
nerc_frac_monthly.loc['WECC'].head()

% generation  % total fuel  % elec fuel
state month type                                         
AZ    1     HYC            1.0           1.0          1.0
      2     HYC            1.0           1.0          1.0
      3     HYC            1.0           1.0          1.0
      4     HYC            1.0           1.0          1.0
      5     HYC            1.0           1.0          1.0

This is it!

In [434]:
nerc_frac_monthly.head()

% generation  % total fuel  % elec fuel
nerc state month type                                         
SERC AL    1     COW            1.0           1.0          1.0
           2     COW            1.0           1.0          1.0
           3     COW            1.0           1.0          1.0
           4     COW            1.0           1.0          1.0
           5     COW            1.0           1.0          1.0

In [423]:
nercs

array(['WECC', 'RFC', 'MRO', 'SERC', '-', 'SPP', 'FRCC', 'NPCC', 'TRE'], dtype=object)

In [ ]:
(nerc_frac_monthly.loc[nerc]['% generation']
 * state_extra_2016.loc[:, 'generation (mwh)']).dropna()

In [443]:
df_list_outer = []
for year in [2016, 2017]:
    df_list_inner = []
    for nerc in nercs:
        df = pd.concat([(nerc_frac_monthly
                         .loc[nerc]['% generation']
                         * state_extra
                         .loc[year]['generation (mwh)']).dropna(),
                        (nerc_frac_monthly.
                         loc[nerc]['% total fuel']
                         * state_extra
                         .loc[year]['total fuel (mmbtu)']).dropna(),
                        (nerc_frac_monthly
                         .loc[nerc]['% elec fuel']
                         * state_extra
                         .loc[year]['elec fuel (mmbtu)']).dropna()],
                        axis=1)
        df.columns = nerc_frac_match.values()
        df['nerc'] = nerc
        df['year'] = year
        df = df.groupby(['year', 'nerc', 'month', 'type']).sum()
        df_list_inner.append(df)

    df_list_outer.append(pd.concat(df_list_inner))
final = pd.concat(df_list_outer)

In [444]:
final

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
year nerc month type                                                         
2016 WECC 1     COW       3.758774e+05        6.325535e+06       4.298811e+06
                GEO       5.326896e+05                 NaN                NaN
                HYC       2.137328e+06                 NaN                NaN
                NG        2.231104e+06        2.732402e+07       1.755353e+07
                OOG      -3.632221e+04                 NaN                NaN
                OTH       5.475943e+04                 NaN                NaN
                PC        0.000000e+00        0.000000e+00       0.000000e+00
                PEL       1.538360e+03        5.864537e+04       1.877487e+04
                SUN       2.913541e+05                 NaN                NaN
                WAS       2.592628e+05                 NaN                NaN
                WND       2.451469e+05                 NaN                NaN
                WWW       2.416416e+05                 NaN                NaN
          2     COW       3.051662e+05        5.529328e+06       3.600548e+06
                GEO       4.955619e+05                 NaN                NaN
                HYC       2.254750e+06                 NaN                NaN
                NG        1.662090e+06        2.052736e+07       1.241505e+07
                OOG      -3.554078e+04                 NaN                NaN
                OTH       4.359569e+04                 NaN                NaN
                PC        0.000000e+00        0.000000e+00       0.000000e+00
                PEL       1.702053e+03        6.981510e+04       1.626148e+04
                SUN       4.657585e+05                 NaN                NaN
                WAS       2.315461e+05                 NaN                NaN
                WND       2.645454e+05                 NaN                NaN
                WWW       2.280932e+05                 NaN                NaN
          3     COW       2.519467e+05        4.855183e+06       2.935199e+06
                GEO       5.317773e+05                 NaN                NaN
                HYC       2.991431e+06                 NaN                NaN
                NG        1.588727e+06        2.065276e+07       1.233239e+07
                OOG      -3.570532e+04                 NaN                NaN
                OTH       4.803096e+04                 NaN                NaN
...                                ...                 ...                ...
2017 TRE  4     HYC       1.032891e+05                 NaN                NaN
                NG        3.361071e+06        3.723031e+07       2.851581e+07
                OOG       3.592197e+04                 NaN                NaN
                OTH       3.620240e+03                 NaN                NaN
                PC        0.000000e+00        1.006100e+04      -4.000000e+00
                PEL       2.303771e+01        2.171757e+02       3.065437e+02
                SUN       9.248117e+04                 NaN                NaN
                WAS       1.533996e+04                 NaN                NaN
                WND       6.339001e+05                 NaN                NaN
                WWW       2.542933e+04                 NaN                NaN
          5     HYC       1.264384e+05                 NaN                NaN
                NG        3.600007e+06        3.985479e+07       3.140188e+07
                OOG       3.692411e+04                 NaN                NaN
                OTH       4.674410e+03                 NaN                NaN
                PC        4.283650e+02        1.966200e+04       5.025000e+03
                PEL       2.383657e+01        2.141016e+02       3.036934e+02
                SUN       1.142562e+05                 NaN                NaN
                WAS       1.606144e+04                 NaN                NaN
                WND       5.626438e+05      

In [445]:
path = join(cwd, '..', 'Data storage', 'NERC extra gen fuels.csv')
final.to_csv(path)